# Traffic Sign Segmentation

**Setup**

In [1]:
## Setup
import sys
# Python 3.7 is required
assert sys.version_info >= (3,7)

# Import Library
import os
import time
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Make sure that optimization is enabled
if not cv.useOptimized():
    cv.setUseOptimized(True)

cv.useOptimized()

True

## Function

### 1) Image Preprocessing
- **Gaussian Filtering**  
 1) To remove noise

In [2]:
def img_preprocessing (image):
    # Variable
    kernel_size = 7
    
    # Make a copy image
    img_copy = image.copy()
    
    # Gaussian filter
    img_gaussian = cv.GaussianBlur(img_copy, (kernel_size,kernel_size), 1)
    
    return img_gaussian

### 2) Largest Contour Detection

In [3]:
def largest_contour_detect(images):
    # Find contours
    _, image = cv.threshold(images, 200, 255, cv.THRESH_TOZERO)
    contours, hierarchy = cv.findContours(image, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    
    # Contour List
    if (len(contours) == 0):
        white_mask = np.zeros(image.shape, dtype = np.uint8)
        return white_mask, white_mask, white_mask
    else:
        cnt_list = np.zeros(len(contours))
        for i in range(0,len(contours)):
            cnt_list[i] = cv.contourArea(contours[i])
        
    # Find largest contour
    largest_contour_index = np.argmax(cnt_list)
    largest_contour = contours[largest_contour_index]
    contour_mask = np.zeros(image.shape, dtype = np.uint8)
    
    # Draw Contour
    if len(contours) != 0:
        cv.drawContours(contour_mask, contours, largest_contour_index, (255,255,255), -1)
        
    return image, largest_contour, contour_mask

### 3) Bounding Box Detection

In [4]:
def bounding_box(image, largest_contour):
    # Make a copy image
    bounding_box = image.copy()
    
    # If cannot find the contour
    if (largest_contour is None):
        return bounding_box, (0,0,0,0)
    
    x,y,w,h = cv.boundingRect(largest_contour)
    cv.rectangle(bounding_box,(x,y),(x+w,y+h),(0,255,0),2)
    
    return bounding_box, (x,y,x+w,y+h)

### 4) Image Segmentation
- **HSV Color segmentation**: To segment out blue color

- **Canny Edge Detection**: To detect the edge of traffic sign in mask

- **Morphological Prpcessing**
    - **Dilation** : To expand inside white region
    - **Erosion** : To decrease outside white region

In [5]:
def img_segmentation(annotation_data, image, traffic_sign, save_res):
    # Variable
    sec = time.time()
    traffic_signs = np.where(annotation_data['Category'].isin(traffic_sign))[0]
    all = np.empty((150,1200,3))
    points = []
    accuracy = [0,0,0,0,0]
    kernel = np.ones((3,3), np.uint8)
    # set Blue range
    blue_low, blue_high = (90, 60, 10), (135, 255, 255)
    
    for i in range (len(traffic_signs)):
        # Copy each image
        image_copy = image[traffic_signs[i]].copy()
        h, w = image_copy.shape[:2]
        
        ## Image Preprocessing
        img= img_preprocessing(image_copy)
    
        # HSV color segmentation
        img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    
        # Blue mask
        hsv_mask = cv.inRange(img_hsv, blue_low, blue_high)
    
        # Canny edge detection
        img_canny = cv.Canny(hsv_mask, 50, 100)
    
        # Morphological Processing
        img_Dil = cv.dilate(img_canny, kernel, iterations = 1)
        img_Ero = cv.erode(img_Dil, kernel, iterations = 1)
    
        # Find largest contour detection
        th, largest_contour, contour_mask = largest_contour_detect(img_Ero)
    
        # Bounding box detection
        img_bounding_box, box_point = bounding_box(img, largest_contour)
    
        points.append(box_point)
    
        # calculating IOU accuracy and storing into different category accuracy group
        area_1 = (box_point[2] - box_point[0]) * (box_point[3] - box_point[1])
        area_2 = (train_annotation['End_X'][i] - train_annotation['Start_X'][i]) * (train_annotation['End_Y'][i] - train_annotation['Start_Y'][i])
        diff_X = max(0, min(box_point[2], train_annotation['End_X'][i]) - max(box_point[0], train_annotation['Start_X'][i]))
        diff_Y = max(0, min(box_point[3], train_annotation['End_Y'][i]) - max(box_point[1], train_annotation['Start_Y'][i]))
        overlap = diff_X * diff_Y
        final_area = area_1 + area_2 - overlap
        accry = round(overlap/final_area*100,2)
        
        if accry >= 95:
            accuracy[0] += 1
        elif accry >= 90:
            accuracy[1] += 1
        elif accry >= 85:
            accuracy[2] += 1
        elif accry >= 80:
            accuracy[3] += 1
        else:
            accuracy[4] += 1
    
        if(save_res):
            hsv_mask = np.stack((hsv_mask,)*3, axis=-1)
            img_canny = np.stack((img_canny,)*3, axis=-1)
            img_Dil = np.stack((img_Dil,)*3, axis=-1)
            img_Ero = np.stack((img_Ero,)*3, axis=-1)
            final_mask = np.stack((contour_mask,)*3, axis=-1)
            result = np.concatenate((image_copy, hsv_mask, img_canny, img_Dil, img_Ero, final_mask, img_bounding_box), axis = 1)
            result = cv.resize(result, (1050,150), interpolation = cv.INTER_CUBIC)
            result = np.concatenate((result, np.zeros((150, 150, 3))), axis = 1)
            result = np.concatenate((np.zeros((30, 1200, 3)), result), axis = 0)
        
            cv.putText(result, ("Original Image"), (5, 15), 0, 0.5, (255,255,255), 1, cv.INTER_CUBIC)
            cv.putText(result, ("HSV Mask"), (155, 15), 0, 0.5, (255,255,255), 1, cv.INTER_CUBIC)
            cv.putText(result, ("Canny Edge"), (305, 15), 0, 0.5, (255,255,255), 1, cv.INTER_CUBIC)
            cv.putText(result, ("Dilation"), (455, 15), 0, 0.5, (255,255,255), 1, cv.INTER_CUBIC)
            cv.putText(result, ("Erosion"), (605, 15), 0, 0.5, (255,255,255), 1, cv.INTER_CUBIC)
            cv.putText(result, ("Contour"), (755, 15), 0, 0.5, (255,255,255), 1, cv.INTER_CUBIC)
            cv.putText(result, ("Bounding Box"), (905, 15), 0, 0.5, (255,255,255), 1, cv.INTER_CUBIC)
            cv.putText(result, ("Accuracy"), (1055, 15), 0, 0.5, (255,255,255), 1, cv.INTER_CUBIC)
            cv.putText(result, str(accry) + "%", (1100, 110), 0, 0.5, (255,255,255), 1, cv.INTER_CUBIC)
                
            all = np.concatenate((all, result), axis = 0)
    
            if (i%100 == 0):
                cv.imwrite('result' + str(i//100) + '.png', all)
                all = result
                print(i, "/", len(traffic_signs), "Elapsed time:", time.time() - sec, "Average time:", (time.time() - sec)/(i+1))
            elif (i%100 == 0):
                print(i, "/", len(traffic_signs), "Elapsed time:", time.time() - sec, "Average time:", (time.time() - sec)/(i+1))
    
    return points, accuracy

## Load Training Dataset

**1) Load TSRD Train Annotation**

In [6]:
# Path to the TSRD train annotation txt file
path = os.path.join(os.getcwd(), 'TSRD-Train Annotation\\TsignRecgTrain4170Annotation.txt')

# Column's names
columns = ['File Name', 'Width', 'Height', 'Start_X', 'Start_Y', 'End_X', 'End_Y', 'Category']

# Read the data of the TSRD train annotation txt file to the train_annotation
train_annotation = pd.read_csv(path, sep=";", index_col = False, names = columns)

**2) Load Train Images**

In [7]:
#path to the folder containing the trains set image files
path = os.path.join(os.getcwd(), 'TSRD-train')

#read the train images into train_image
train_image = []
for i in range (train_annotation.shape[0]):
    train_image.append(cv.imread(path + '\\' + train_annotation.iloc[i][0]))

## Segmentation Process

In [8]:
# All traffic signs values
ts_cat = tuple(range(0, 58))

points, accuracy = img_segmentation(train_annotation, train_image, ts_cat, save_res = True)

0 / 4170 Elapsed time: 0.02698373794555664 Average time: 0.02698373794555664
100 / 4170 Elapsed time: 12.092069864273071 Average time: 0.11972346400270367
200 / 4170 Elapsed time: 22.895881414413452 Average time: 0.1139098577831515
300 / 4170 Elapsed time: 33.6307315826416 Average time: 0.11173000525794552
400 / 4170 Elapsed time: 44.58791995048523 Average time: 0.11119182032539958
500 / 4170 Elapsed time: 55.33595156669617 Average time: 0.11045100113113007
600 / 4170 Elapsed time: 66.17390203475952 Average time: 0.11010632618096426
700 / 4170 Elapsed time: 76.96869969367981 Average time: 0.10979843037614809
800 / 4170 Elapsed time: 87.99837946891785 Average time: 0.10986064852549045
900 / 4170 Elapsed time: 98.70924496650696 Average time: 0.10955521083963037
1000 / 4170 Elapsed time: 109.348149061203 Average time: 0.10923891015105196
1100 / 4170 Elapsed time: 120.18194270133972 Average time: 0.10915707783954562
1200 / 4170 Elapsed time: 130.86782121658325 Average time: 0.1089657129197

## Evaluation

In [9]:
## Evaluation result
overlap_total = 0
final_area = 0

for i in range (len(points)):
    area_1 = (points[i][2] - points[i][0]) * (points[i][3] - points[i][1])
    area_2 = (train_annotation['End_X'][i] - train_annotation['Start_X'][i]) * (train_annotation['End_Y'][i] - train_annotation['Start_Y'][i])
    diff_X = max(0, min(points[i][2], train_annotation['End_X'][i]) - max(points[i][0], train_annotation['Start_X'][i]))
    diff_Y = max(0, min(points[i][3], train_annotation['End_Y'][i]) - max(points[i][1], train_annotation['Start_Y'][i]))
    overlap = diff_X * diff_Y
    overlap_total += overlap
    final_area = final_area + area_1 + area_2 - overlap

print("Total Images:", str(accuracy[0] + accuracy[1] + accuracy[2] + accuracy[3] + accuracy[4]))
print("Overall Accuracy of Segmentation:", overlap_total/final_area)
print("Number of 95% - 100% Accuracy:", str(accuracy[0]))
print("Number of 90% - 95% Accuracy:", str(accuracy[1]))
print("Number of 85% - 90% Accuracy:", str(accuracy[2]))
print("Number of 80% - 85% Accuracy:", str(accuracy[3]))
print("Number of lower than 80% Accuracy:", str(accuracy[4]))

Total Images: 4170
Overall Accuracy of Segmentation: 0.3030884739095587
Number of 95% - 100% Accuracy: 574
Number of 90% - 95% Accuracy: 270
Number of 85% - 90% Accuracy: 52
Number of 80% - 85% Accuracy: 50
Number of lower than 80% Accuracy: 3224


## Conclusion
The total overall is quite not good, this is because we only focus on blue color traffic signs. However, we have successfully segment more than 946 blue color traffic sign images and it is achieved our objective which is more than 70 images had 80 percent accuracy. In addition, we have 574 images is achieved more than 95 percent accuracy.